In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
from collections import OrderedDict
from django.db.models import Avg, Max

measures = OrderedDict()

measures['I-732 (carbon tax)'] = {'yes': ['CARBW  103'], 'no': ['NO732  507', 'NORTPP 501']}
measures['I-735 (constitutional amendment)'] = {'yes': ['FIX DF 103', 'WASTCA 166'], 'no': []}
measures['I-1433 (minimum wage)'] = {'yes': ['RAISUW 101'], 'no': ['NO  I1 507']}
measures['I-1464 (campaign finance)'] = {'yes': ['INTEW  109'], 'no': ['NO1464 005', 'COALSC 107']}
measures['I-1491 (gun control)'] = {'yes': ['ALLIGR 111'], 'no': ['KN1491 052']}
measures['I-1501 (senior ID theft)'] = {'yes': ['CAMPPF 111'], 'no': []}
measures['I-1515 (transgender bathrooms)'] = {'yes': ['JUSTWP 012'], 'no': ['WASHWD 111']}
measures['ST3'] = {'yes': ['MASSTN 101'], 'no': ['NO  ST 034', 'FAMIST 009']}
measures['Seattle-124 (hotel workers)'] = {'yes': ['SEATPW 121'], 'no': ['COMMEA 501']}

zipcode_funding = {}

for measure in measures:
    for idx, side in enumerate(['yes', 'no']):
        print 'Ballot measure: %s, %s' % (measure, side.title())
        contributions = ScrapedContribution.objects.filter(filing_committee__filer_id__in=measures[measure][side])
        if not contributions:
            print 'No contributions on file'
        else:
            print locale.format('%d', contributions.count(), grouping=True), 'contributions'
            donors = contributions.values('donor', 'city', 'state').distinct()
            print locale.format('%d', donors.count(), grouping=True), 'donors'
            print locale.currency(contributions.aggregate(Sum('amount'))['amount__sum'], grouping=True), 'raised'
            print locale.currency(
                contributions.aggregate(Avg('amount'))['amount__avg'], grouping=True), 'average contribution'
            print locale.currency(
                contributions.aggregate(Max('amount'))['amount__max'], grouping=True), 'is the largest contribution'
            zipcodes = contributions.values('zipcode').annotate(
                sum=Sum('amount'), num=Count('date')).order_by('-sum')
            print 'Top 5 zipcodes:'
            for zipcode in zipcodes[:5]:    
                print '%s: %s in %d contributions' % (
                    zipcode['zipcode'], locale.currency(zipcode['sum'], grouping=True), zipcode['num'])
                if zipcode['zipcode'] in unemployment_results:
                    print 'Unemployment rate for this zipcode is %.2f%%' % unemployment_results[zipcode['zipcode']]
            for zipcode in zipcodes:
                zc = zipcode['zipcode'][:5]
                if zc not in zipcode_funding:
                    zipcode_funding[zc] = {
                        'I-732 (carbon tax)': {'yes': 0., 'no': 0.},
                        'I-735 (constitutional amendment)': {'yes': 0., 'no': 0.},
                        'I-1433 (minimum wage)': {'yes': 0., 'no': 0.},
                        'I-1464 (campaign finance)': {'yes': 0., 'no': 0.},
                        'I-1491 (gun control)': {'yes': 0., 'no': 0.},
                        'I-1501 (senior ID theft)': {'yes': 0., 'no': 0.},
                        'I-1515 (transgender bathrooms)': {'yes': 0., 'no': 0.},
                        'ST3': {'yes': 0., 'no': 0.},
                        'Seattle-124 (hotel workers)': {'yes': 0., 'no': 0.},
                        'unemployment': -1.,
                        'percent_white': -1.,
                        'percent_black': -1.,
                        'percent_asian': -1.,
                        'percent_hispanic': -1.,
                        'median_age': -1.
                    }
                zipcode_funding[zc][measure][side] += float(zipcode['sum'])
        if idx < 1:
            print '\n'
    print '-------\n'

Ballot measure: I-732 (carbon tax), Yes
2,886 contributions
1,483 donors
$1,555,653.99 raised
$539.03 average contribution
$100,000.00 is the largest contribution
Top 5 zipcodes:
98040: $173,345.00 in 96 contributions


NameError: name 'unemployment_results' is not defined